# 循环神经网络 Recurrent Neural Networks

## 1. 为什么要使用序列模型 Why sequence models

![Examples of sequence data.png](img/Examples of sequence data.png)

## 2. 标记符号 Notations

以命名实体识别（Named Entity Recognition）问题为例，我要需要一套标记符号，来描述这个监督学习问题。这里，用符号 $x^{(i)<t>}$ 表示第 i 个训练集的第 t 个词，用符号 $T_x^{(i)}$ 表示第 i 个训练集总共的单词数。目标变量 y 也类似。

![Motivating example RNN.png](img/Motivating example RNN.png)

由于这里是一个NLP（Natural Language Processing）问题，就涉及到如何来表示单词。通常我们会构造一个词典，以30000~50000的单词量为最常见，也有100000级别的词典，甚至在一些互联网公司，百万级别的词典也不是没有。有了词典之后，每个单词就会有一个序号。而将所有单词进行One-Hot编码，就可以获得表示这个单词的向量。

![Representing words.png](img/Representing words.png)

## 3. 循环神经网络模型 Recurrent Neural Network Model

使用标准神经网络模型来处理序列数据，主要会面临以下两个问题：1）序列数据的输入输出长度，对于不同的样本，可以是不同的；2）标准神经网络不会共享对序列数据的位置相关特征。

![Why not a standard network.png](img/Why not a standard network.png)

循环神经网络的模型：初始化 $a^{<0>}$ 为0向量，红色的参数共享，序列中每个单词的激活结果，作为下一个单词模型的输入之一。下图的左右描述是等价的。注意到这个RNN模型实际上是单向RNN，顺序在前的单词可以对顺序在后的单词施加影响，反之则不行，因而模型也就无法区分下面两个句子的不同。后面会说，经过一点简单的修改，也可以获得**双向RNN（Bidirected RNN）**

![Recurrent Neural Networks.png](img/Recurrent Neural Networks.png)

RNN的前向传播过程：

![Forward Propagation.png](img/Forward Propagation.png)

部分参数可以合并，从而简化计算：

![Simplified RNN notation.png](img/Simplified RNN notation.png)